# How to get MOC data from VESPA in Python

Illustrate search for TAP MOC for vespa using 'pyvo' and 'astropy'

## Import required modules

In [1]:
import pyvo as vo
from mpl_toolkits.basemap import Basemap

## Search a specific TAP service

We are looking for VVEx - VIRTIS/Venus Express database since it has a MOC

### Search the database containing this service

In [2]:
keywords = ["Venus"]

In [3]:
databasesList = vo.registry.search(servicetype='tap',keywords=keywords, includeaux=True)

In [4]:
for d in databasesList :
    print(d.short_name)

J/A+A/537/L2
J/A+A/585/A53
J/A+A/606/A53
J/A+A/613/A25
J/A+A/619/A99
J/A+A/627/A82
J/A+A/653/A41
J/AJ/154/207
J/AJ/158/196
J/AJ/160/131
J/AJ/160/259
J/AJ/162/216
J/ApJ/800/135
J/ApJ/807/45
J/ApJ/893/L27
J/MNRAS/462/4300
J/MNRAS/485/3999
J/MNRAS/519/6210
VI/127

GRB/IPN
KONUS
Cassini RPWS
BDIP
bdip.observatori
HST Planeto
VVEx
MESS-MAG-VSO
MESS-MAG-VSO-AVG
PVO-PHEM-ASC
PVO-PHEM-BIN
VEX-ASPERA4-DER
VEX-ASPERA4-ENG
VEX-ASPERA4-RAW
VEX-HIGH-MAG


### Search the services in the databases that were found

In [5]:
urlsList = set(databasesList.to_table().field("access_urls"))

In [6]:
tables = []

for url in urlsList :
    print("Searching in service " + url)
    service = vo.dal.TAPService(url)
    searchResult = service.search("SELECT schema_name, table_name FROM tap_schema.tables WHERE schema_name LIKE '%"+keywords[0]+"%' OR table_name LIKE '%"+keywords[0]+"%' OR description LIKE '%"+keywords[0]+"%'")
    for tableFound in searchResult["table_name"] :
        tables.append({"url":url,"name":tableFound})
print("DONE")

Searching in service https://vo-pds-ppi.igpp.ucla.edu/tap
Searching in service https://heasarc.gsfc.nasa.gov/xamin/vo/tap
Searching in service http://voparis-tap-maser.obspm.fr/tap
Searching in service http://tapvizier.cds.unistra.fr/TAPVizieR/tap
Searching in service http://vo.lmd.jussieu.fr/tap
Searching in service http://voparis-tap-planeto.obspm.fr/tap
DONE


In [7]:
for table in tables :
    print(table["name"]," at ", table["url"])

vex_cleaned_high_res_mag.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
vex_aspera4_els_raw.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
mess_mag_calibrated_vso.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
vex_aspera4_els_derived.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
mess_mag_calibrated_vso_avg.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
pvo_ephem_vso_asc.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
vex_aspera4_els_eng.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
pvo_ephem_vso_bin.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
J/A+A/627/A82/tablea1  at  http://tapvizier.cds.unistra.fr/TAPVizieR/tap
J/A+A/650/A156/tablec1  at  http://tapvizier.cds.unistra.fr/TAPVizieR/tap
J/A+A/627/A82/tablea3  at  http://tapvizier.cds.unistra.fr/TAPVizieR/tap
J/A+A/650/A156/table2  at  http://tapvizier.cds.unistra.fr/TAPVizieR/tap
J/A+A/537/L2/tablea1  at  http://tapvizier.cds.unistra.fr/TAPVizieR/tap
J/A+A/627/A82/tablea2  at  http://tapvizier.cds.un

## Only keep tables with a 'coverage' key (and so maybe a MOC)

In [8]:
def doContainMOC(table) :
    service = vo.dal.TAPService(table["url"])
    try :
        searchResult = service.search("SELECT TOP 1 * FROM "+table["name"]+" WHERE coverage IS NOT NULL")
        if searchResult == "" :
            return False
        else :
            return True
    except :
        return False

In [9]:
tablesWithMOC = []
for table in tables :
    print("Checking table ",table["name"])
    if doContainMOC(table) :
        tablesWithMOC.append(table)

Checking table  vex_cleaned_high_res_mag.epn_core
Checking table  vex_aspera4_els_raw.epn_core
Checking table  mess_mag_calibrated_vso.epn_core
Checking table  vex_aspera4_els_derived.epn_core
Checking table  mess_mag_calibrated_vso_avg.epn_core
Checking table  pvo_ephem_vso_asc.epn_core
Checking table  vex_aspera4_els_eng.epn_core
Checking table  pvo_ephem_vso_bin.epn_core
Checking table  J/A+A/627/A82/tablea1
Checking table  J/A+A/650/A156/tablec1
Checking table  J/A+A/627/A82/tablea3
Checking table  J/A+A/650/A156/table2
Checking table  J/A+A/537/L2/tablea1
Checking table  J/A+A/627/A82/tablea2
Checking table  J/AJ/162/216/fig11
Checking table  vvex.epn_core


In [10]:
for table in tablesWithMOC :
    print(table["name"])

vvex.epn_core


## Display the first 10 elements in the table
Here we only consider the first table found in the previous section

In [11]:
def getData(tapService, serviceStructure, query) :
    if doContainMOC(serviceStructure) :
        return tapService.search(query).to_table()
    else :
        return "NO MOC DATA"

In [12]:
table = tablesWithMOC[0]
tapService = vo.dal.TAPService(table["url"])

data = tapService.search("SELECT TOP 10 * FROM " + table["name"]).to_table()
print(data)

granule_uid granule_gid   obs_id  ... science_case_id sc_pointing_mode
                                  ...                                 
----------- ----------- --------- ... --------------- ----------------
 VV0135_08G    geometry VV0135_08 ...               2           MOSAIC
 VV0134_12C  calibrated VV0134_12 ...               2           MOSAIC
 VV0134_12G    geometry VV0134_12 ...               2           MOSAIC
 VV0134_11C  calibrated VV0134_11 ...               2           MOSAIC
 VV0134_11G    geometry VV0134_11 ...               2           MOSAIC
 VV0134_01C  calibrated VV0134_01 ...               2           MOSAIC
 VV0134_01G    geometry VV0134_01 ...               2           MOSAIC
 VV0134_09C  calibrated VV0134_09 ...               2           MOSAIC
 VV0134_09G    geometry VV0134_09 ...               2           MOSAIC
 VV0134_07G    geometry VV0134_07 ...               2           MOSAIC


## Import modules for Aladin and MOCs

In [13]:
import astropy.units as u
from astropy.coordinates import SkyCoord

from astroquery.cds import cds
from astroquery.vizier import Vizier

from mocpy import MOC

from ipyaladin import Aladin

## Get the data

In [14]:
table = tablesWithMOC[0]
tapService = vo.dal.TAPService(table["url"])

data = tapService.search("SELECT TOP 500 * FROM " + table["name"] + " WHERE coverage IS NOT NULL").to_table()
print(data)

granule_uid granule_gid   obs_id  ... science_case_id sc_pointing_mode
                                  ...                                 
----------- ----------- --------- ... --------------- ----------------
 VV0135_08G    geometry VV0135_08 ...               2           MOSAIC
 VV0134_12C  calibrated VV0134_12 ...               2           MOSAIC
 VV0134_12G    geometry VV0134_12 ...               2           MOSAIC
 VV0134_11C  calibrated VV0134_11 ...               2           MOSAIC
 VV0134_11G    geometry VV0134_11 ...               2           MOSAIC
 VV0134_01C  calibrated VV0134_01 ...               2           MOSAIC
        ...         ...       ... ...             ...              ...
 VV0265_06C  calibrated VV0265_06 ...               2        NADIR_POW
 VV0265_06G    geometry VV0265_06 ...               2        NADIR_POW
 VV0265_05C  calibrated VV0265_05 ...               2        NADIR_POW
 VV0265_05G    geometry VV0265_05 ...               2        NADIR_POW
 VV026

## Load Aladin and add the data that was fetched just above

Here we are using the hips of Venus

In [30]:
aladin = Aladin(
    coo_frame=data[0]["spatial_frame_type"],
    survey="http://voparis-srv-paris.obspm.fr/vo/planeto/hips/CDS_P_Venus_Magellan_C3-MDIR-2025m/"
)
aladin

Aladin(coo_frame='body', options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log', 'overlay_sur…

In [31]:
aladin.add_table(data)

## Load the first MOC data

It needs to be converted to JSON

In [32]:
mocData = tapService.search("SELECT TOP 1 coverage FROM " + table["name"]).to_table()[0]["coverage"]

In [33]:
moc = MOC.from_str(mocData)

In [34]:
mocCenter = moc.barycenter()

In [35]:
jsonMoc = moc.serialize(format='json', optional_kw_dict=None, pre_v2=False)
aladin.add_moc_from_dict(jsonMoc, {'color' : 'red', 'opacity' : 0.5})
aladin.target = mocCenter.to_string()

## Intersection of MOCs

In [85]:
table = tablesWithMOC[0]
tapService = vo.dal.TAPService(table["url"])
query = 'SELECT * FROM '+table["name"]+' WHERE c1min >= 0.0 AND c1max <= 360.0 AND c2min >= 0.0 AND c2max <= 90.0 AND dataproduct_type LIKE \'%sc%\' AND local_time_min <= 8 AND local_time_max >= 9'
#query = 'SELECT granule_uid, c1min,c2min FROM '+table["name"]+' WHERE channel_id LIKE \'%VIRTIS_H%\' AND c1max >= 0.0 AND c1min <= 360.0 AND c2max >= 0.0 AND c2min <= 90.0 AND dataproduct_type LIKE \'%sc%\' AND local_time_min <= 8 AND local_time_max >= 9'

data = tapService.search(query).to_table()

In [86]:
print("granule_uid\tchannel_id\tcoverage c1min\t\tc1max\t\tc2min\t\tc2max")
for e in data :
    print(e["granule_uid"],"\t",e["channel_id"],"\t", e["coverage"][0:5], "\t", e["c1min"], "\t", e["c1max"], "\t", e["c2min"], "\t", e["c2max"])

granule_uid	channel_id	coverage c1min		c1max		c2min		c2max
VV0047_00C 	 VIRTIS_M_VIS 	 6/107 	 187.423 	 3.6804 	 4.82867 	 89.6728
VV0043_00C 	 VIRTIS_M_VIS 	 7/445 	 194.289 	 349.923 	 999.0 	 -999.0
VT0033_00C 	 VIRTIS_H 	 7/108 	 165.763 	 345.301 	 6.09572 	 89.6878
VT0043_00C 	 VIRTIS_H 	 8/173 	 191.872 	 341.843 	 999.0 	 -999.0
VI0043_00C 	 VIRTIS_M_IR 	 7/445 	 194.288 	 349.923 	 999.0 	 -999.0
VI0047_00C 	 VIRTIS_M_IR 	 6/107 	 187.422 	 3.6675 	 4.79193 	 89.6692


In [87]:
colors = ["red","blue","green"]

In [88]:
mocGroups = {"VIRTIS_M_VIS":[], "VIRTIS_M_IR":[], "VIRTIS_H":[]}
for element in data :
    mocGroups[element["channel_id"]].append(MOC.from_str(element["coverage"]))

keys = list(mocGroups.keys())
for key in keys:
    if(len(mocGroups[key]) > 0):
        mocUnion = mocGroups[key][0]
        mocGroups[key].pop(0)
        for moc in mocGroups[key] :
            mocUnion = mocUnion.union(moc)
        mocGroups[key] = mocUnion
    else :
        mocGroups.pop(key)

In [89]:
aladin = Aladin(
    coo_frame=data[0]["spatial_frame_type"],
    survey="http://voparis-srv-paris.obspm.fr/vo/planeto/hips/CDS_P_Venus_Magellan_C3-MDIR-2025m/"
)
aladin

Aladin(coo_frame='body', options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log', 'overlay_sur…

In [90]:
aladin.add_table(data)

In [91]:
i = 0
for key in mocGroups.keys() :
    moc = mocGroups[key]
    jsonMoc = moc.serialize(format='json', optional_kw_dict=None, pre_v2=False)
    aladin.add_moc_from_dict(jsonMoc, {'color' : colors[i], 'opacity' : 0.5, 'name' : key})
    i = i + 1
aladin.target = mocCenter.to_string()

In [92]:
# Get one of the moc to use it to center the view in Aladin
mocCenter = next(iter(mocGroups.values())).barycenter()